In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Option 1: Add project root to sys.path
import sys
from pathlib import Path

# Go up two levels from notebooks/centralized to reach project root
project_root = str(Path().absolute().parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import from src normally
from src.common.client_utils import load_femnist_dataset

/home/lcm76/.conda/envs/fl-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-18 21:44:29,381	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
from pathlib import Path
from typing import Any
from logging import INFO, DEBUG
import json
import os
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from scipy.signal import medfilt
from flwr.common import log, ndarrays_to_parameters
import matplotlib.pyplot as plt
import math

from src.common.client_utils import (
    load_femnist_dataset,
    get_network_generator_cnn as get_network_generator,
    get_device,
    get_model_parameters,
    aggregate_weighted_average,
)


from src.flwr_core import (
    set_all_seeds,
    get_paths,
    decompress_dataset,
    get_flower_client_generator,
    sample_random_clients,
    get_federated_evaluation_function,
)

from src.estimate import (
    compute_critical_batch,
)

from src.experiments_simulation import (
    run_simulation,
    centralized_experiment,
)

from src.utils import get_centralized_acc_from_hist

PathType = Path | str | None

In [4]:
set_all_seeds()

PATHS = get_paths()

HOME_DIR = PATHS["home_dir"]
DATASET_DIR = PATHS["dataset_dir"]
DATA_DIR = PATHS["data_dir"]
CENTRALIZED_PARTITION = PATHS["centralized_partition"]
CENTRALIZED_MAPPING = PATHS["centralized_mapping"]
FEDERATED_PARTITION = PATHS["federated_partition"]

# extract dataset from tar.gz
decompress_dataset(PATHS)

In [5]:
NETWORK_GENERATOR = get_network_generator()
SEED_NET = NETWORK_GENERATOR()
SEED_MODEL_PARAMS = get_model_parameters(SEED_NET)
CID_CLIENT_GENERATOR = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_PARTITION)

In [6]:
# Federated configuration dictionary
federated_train_config = {
    "epochs": 10,
    "batch_size": 32,
    "client_learning_rate": 0.01,
    "weight_decay": 0,
    "num_workers": 0,
    "max_batches": 100,
}

federated_test_config: dict[str, Any] = {
    "batch_size": 32,
    "num_workers": 0,
    "max_batches": 100,
}

num_rounds = 10
num_total_clients = 100
num_evaluate_clients = 0
num_clients_per_round = 10

initial_parameters = ndarrays_to_parameters(SEED_MODEL_PARAMS)

federated_evaluation_function = get_federated_evaluation_function(
    batch_size=federated_test_config["batch_size"],
    num_workers=federated_test_config["num_workers"],
    model_generator=NETWORK_GENERATOR,
    criterion=nn.CrossEntropyLoss(),
    max_batches=None if "max_batches" not in federated_test_config else federated_test_config["max_batches"],
)

server_learning_rate = 1.0
server_momentum = 0.0
accept_failures = False


CID_CLIENT_GENERATOR = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_PARTITION)

list_of_ids = sample_random_clients(
    num_total_clients, federated_train_config["batch_size"],
    CID_CLIENT_GENERATOR,
)

federated_client_generator = (
    get_flower_client_generator(
        NETWORK_GENERATOR, FEDERATED_PARTITION, lambda seq_id: list_of_ids[seq_id]
    )
)

INFO flwr 2025-03-18 21:44:32,319 | flwr_core.py:107 | cid: 2530


INFO flwr 2025-03-18 21:44:32,400 | flwr_core.py:107 | cid: 2184
INFO flwr 2025-03-18 21:44:32,404 | flwr_core.py:107 | cid: 2907
INFO flwr 2025-03-18 21:44:32,406 | flwr_core.py:107 | cid: 1498
INFO flwr 2025-03-18 21:44:32,408 | flwr_core.py:107 | cid: 2338
INFO flwr 2025-03-18 21:44:32,411 | flwr_core.py:107 | cid: 2399
INFO flwr 2025-03-18 21:44:32,413 | flwr_core.py:107 | cid: 2997
INFO flwr 2025-03-18 21:44:32,415 | flwr_core.py:107 | cid: 678
INFO flwr 2025-03-18 21:44:32,418 | flwr_core.py:107 | cid: 3175
INFO flwr 2025-03-18 21:44:32,421 | flwr_core.py:107 | cid: 1363
INFO flwr 2025-03-18 21:44:32,423 | flwr_core.py:107 | cid: 1571
INFO flwr 2025-03-18 21:44:32,425 | flwr_core.py:107 | cid: 2600
INFO flwr 2025-03-18 21:44:32,427 | flwr_core.py:107 | cid: 1473
INFO flwr 2025-03-18 21:44:32,429 | flwr_core.py:107 | cid: 1260
INFO flwr 2025-03-18 21:44:32,433 | flwr_core.py:107 | cid: 1603
INFO flwr 2025-03-18 21:44:32,435 | flwr_core.py:107 | cid: 2855
INFO flwr 2025-03-18 21:44

In [7]:
import pickle

def save_experiment(save_file_name, batch_size, parameters_for_each_round, hist):
    results_dict = {
        'batch_size': batch_size,
        'parameters_for_each_round': parameters_for_each_round,
        'history': hist
    }
    
    with open(save_file_name, 'wb') as f:  # Note: 'wb' for binary write mode
        pickle.dump(results_dict, f)

def load_experiment(file_name):
    with open(file_name, 'rb') as f:  # Note: 'rb' for binary read mode
        results_dict = pickle.load(f)
    
    return (
        results_dict['batch_size'],
        results_dict['parameters_for_each_round'],
        results_dict['history'],
    )

In [ ]:
total_batch_results = []

experiment_batch_sizes = [256] # [16, 32, 64, 128, 256]

for batch_size in experiment_batch_sizes:
    train_cfg = federated_train_config.copy()
    train_cfg["batch_size"] = batch_size
    ratio = np.sqrt(batch_size / 256)
    learning_rate = ratio * 0.01
    train_cfg["client_learning_rate"] = learning_rate # Same as centralized, but should be lower for FL

    test_cfg = federated_test_config.copy()
    test_cfg["batch_size"] = batch_size

    local_list_of_ids = sample_random_clients(num_total_clients, train_cfg["batch_size"], CID_CLIENT_GENERATOR)
    local_federated_client_generator = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_PARTITION, lambda seq_id: local_list_of_ids[seq_id])

    parameters_for_each_round, hist = run_simulation(
        num_rounds = num_rounds,
        num_total_clients = num_total_clients,
        num_clients_per_round = num_clients_per_round,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = num_clients_per_round,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = local_federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        target_accuracy=0.70,
        use_target_accuracy=True,
        )

    total_batch_results.append((batch_size, parameters_for_each_round, hist))
    save_experiment(f"federated_batch_results_{batch_size}.pkl", batch_size, parameters_for_each_round, hist)

INFO flwr 2025-03-18 21:44:32,731 | flwr_core.py:107 | cid: 2530
INFO flwr 2025-03-18 21:44:32,734 | flwr_core.py:107 | cid: 2184
INFO flwr 2025-03-18 21:44:32,736 | flwr_core.py:107 | cid: 2907
INFO flwr 2025-03-18 21:44:32,739 | flwr_core.py:107 | cid: 1498
INFO flwr 2025-03-18 21:44:32,741 | flwr_core.py:107 | cid: 2338
INFO flwr 2025-03-18 21:44:32,743 | flwr_core.py:107 | cid: 2399
INFO flwr 2025-03-18 21:44:32,745 | flwr_core.py:107 | cid: 2997
INFO flwr 2025-03-18 21:44:32,748 | flwr_core.py:107 | cid: 678


INFO flwr 2025-03-18 21:44:32,750 | flwr_core.py:107 | cid: 3175
INFO flwr 2025-03-18 21:44:32,753 | flwr_core.py:107 | cid: 1363
INFO flwr 2025-03-18 21:44:32,755 | flwr_core.py:107 | cid: 1571
INFO flwr 2025-03-18 21:44:32,757 | flwr_core.py:107 | cid: 2600
INFO flwr 2025-03-18 21:44:32,759 | flwr_core.py:107 | cid: 1473
INFO flwr 2025-03-18 21:44:32,761 | flwr_core.py:107 | cid: 1260
INFO flwr 2025-03-18 21:44:32,764 | flwr_core.py:107 | cid: 1603
INFO flwr 2025-03-18 21:44:32,765 | flwr_core.py:107 | cid: 2855
INFO flwr 2025-03-18 21:44:32,767 | flwr_core.py:107 | cid: 839
INFO flwr 2025-03-18 21:44:32,769 | flwr_core.py:107 | cid: 3119
INFO flwr 2025-03-18 21:44:32,771 | flwr_core.py:107 | cid: 2688
INFO flwr 2025-03-18 21:44:32,773 | flwr_core.py:107 | cid: 1494
INFO flwr 2025-03-18 21:44:32,776 | flwr_core.py:107 | cid: 447
INFO flwr 2025-03-18 21:44:32,779 | flwr_core.py:107 | cid: 1742
INFO flwr 2025-03-18 21:44:32,782 | flwr_core.py:107 | cid: 2601
INFO flwr 2025-03-18 21:44:

### Estimate time taken per round

In [ ]:
times_per_round = []
for batch_size in experiment_batch_sizes:
    cohort_size = 2
    train_cfg = federated_train_config.copy()
    ratio = np.sqrt(cohort_size / 100)
    train_cfg["client_learning_rate"] = ratio * 0.01

    test_cfg = federated_test_config.copy()

    parameters_for_each_round, hist = run_simulation(
        num_rounds = 1,
        num_total_clients = num_total_clients,
        num_clients_per_round = cohort_size,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = cohort_size,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        )

    avg_times_per_round = []
    for (round_idx, round_metrics), metrics_acc in zip(hist.metrics_distributed_fit['training_time'], hist.metrics_centralized['accuracy']):
        round_times = [t for _, t in round_metrics['all']]
        avg_time_per_round = np.max(round_times)    
        avg_times_per_round.append(avg_time_per_round)

    time_per_round = np.mean(avg_times_per_round)
    print(time_per_round)
    times_per_round.append((batch_size, time_per_round))

print(times_per_round)